In [10]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

In [37]:
def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options
    print(exps)

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.concat([pd.DataFrame.from_records(opt.calls), pd.DataFrame.from_records(opt.puts)])
        opt['expirationDate'] = e
        options = pd.concat([options, pd.DataFrame.from_records(opt)])

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    # options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    # NOT EXPERIENCING THIS ERROR 08/07/23   
    options['expirationDate'] = pd.to_datetime(options['expirationDate'])
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options

In [38]:
test_data = options_chain("TSLA")

('2023-08-11', '2023-08-18', '2023-08-25', '2023-09-01', '2023-09-08', '2023-09-15', '2023-09-22', '2023-10-20', '2023-11-17', '2023-12-15', '2024-01-19', '2024-03-15', '2024-06-21', '2024-09-20', '2025-01-17', '2025-06-20', '2025-12-19')


In [39]:
test_data

,contractSymbol,strike,bid,ask,volume,openInterest,impliedVolatility,inTheMoney,expirationDate,dte,CALL,mark
0,TSLA230811C00020000,20.0,230.25,230.50,87.0,12.0,10.304691,True,2023-08-11,3,True,230.375
1,TSLA230811C00050000,50.0,199.00,201.85,86.0,49.0,6.570314,True,2023-08-11,3,True,200.425
2,TSLA230811C00085000,85.0,163.95,166.65,18.0,31.0,4.298833,True,2023-08-11,3,True,165.300
3,TSLA230811C00100000,100.0,148.95,151.45,2.0,33.0,3.546876,True,2023-08-11,3,True,150.200
4,TSLA230811C00105000,105.0,144.05,146.90,13.0,13.0,3.703126,True,2023-08-11,3,True,145.475
...,...,...,...,...,...,...,...,...,...,...,...,...
149,TSLA251219P00520000,520.0,267.60,275.50,15.0,12.0,0.324180,True,2025-12-19,864,False,271.550
150,TSLA251219P00530000,530.0,277.25,285.00,24.0,24.0,0.322974,True,2025-12-19,864,False,281.125
151,TSLA251219P00540000,540.0,286.90,294.55,5.0,63.0,0.321906,True,2025-12-19,864,False,290.725
152,TSLA251219P00550000,550.0,296.10,304.10,1.0,82.0,0.320228,True,2025-12-19,864,False,300.100
